In [12]:
import requests, json, glob, os
import dotenv
from queries import getReq, getOne
import pandas as pd

In [4]:
dotenv.load_dotenv()
username = os.getenv("API_username")
password = os.getenv("API_password")
URL = os.getenv("URL")

In [20]:
regenerate_answers = False

# Searching for roles

In [22]:
KW = ["digital","data", "data engineering", "software developer","machine learning","artificial intelligence", "data management", "information management", "digital consultancy"]
for kw in KW:
    kwn = kw.replace(" ","-")
    for k in range(30):
        FILENAME = "data/lists/list_"+kwn+"_"+str(k)+".json"
        if not os.path.exists(FILENAME) or regenerate_answers:
            r = requests.post(URL, # save the result to examine later
                auth=(username, password), # you can pass this without constructor
                json=getReq(kw, 20, k*20)) # no need to json.dumps or add the header manually!
            kw = kw.replace(" ","-")
            if not r.text == 'Unauthorized':
                with open(FILENAME,"w") as f:
                    f.write(r.text)

In [23]:
IDs = []
for file in glob.glob("data/lists/*.json"):
    #print(file)
    with open(file,"r") as f:
        t = f.read()
    if not t == 'Unauthorized':
        D = json.loads(t)
        for x in D["hits"]["hits"]:
            IDs.append(x["_source"]["pageVersionId"])
            #print(x["_source"]["title"])
print(len(IDs),len(set(IDs)))

# Getting individual roles

In [25]:
for k in IDs:
    REQ = getOne(k)
    FILENAME = "data/jobs/job_"+str(k)+".json"
    if not os.path.exists(FILENAME):
        print("Processing ",k)
        r = requests.post(URL, # save the result to examine later
        auth=(username, password), # you can pass this without constructor
        json=REQ) # no need to json.dumps or add the header manually!
        if not r.text == 'Unauthorized':
            with open(FILENAME,"w") as f:
                f.write(r.text)

Processing  5538579
Processing  5528016
Processing  5539452
Processing  5537057
Processing  5539475
Processing  5528411
Processing  5530109
Processing  5530076
Processing  5536504
Processing  5538551
Processing  5539254
Processing  5510644
Processing  5525509
Processing  5525510
Processing  5539215
Processing  5539229
Processing  5538820
Processing  5537551
Processing  5522871
Processing  5535618
Processing  5519757
Processing  5535438
Processing  5535677
Processing  5539244
Processing  5482168
Processing  5483197
Processing  5496793
Processing  5515716
Processing  5533740
Processing  5533929
Processing  5534463
Processing  5535073
Processing  5537526
Processing  5538620
Processing  5538838
Processing  5539373
Processing  5539378
Processing  5486297
Processing  5502916
Processing  5519770
Processing  5527682
Processing  5527679
Processing  5529949
Processing  5533942
Processing  5536832
Processing  5538882
Processing  5533976
Processing  5539220
Processing  5506108
Processing  5521642


In [30]:
IDs = []
F = glob.glob("data/jobs/*.json")
print(len(F))
for file in F:
    with open(file,"r") as f:
        t = f.read()
    if not t == 'Unauthorized':
        D = json.loads(t)
        for x in D["hits"]["hits"]:
            IDs.append(x["_source"])

1387


# Assembling data

In [31]:
def getSkill(x):
    x = x.lower()
    if "data" in x:
        return "Data"
    elif "information management" in x:
        return "Information Management"
    elif "digital" in x:
        return "Digital"
    else:
        return ""


In [33]:
df = pd.DataFrame(IDs)[["jobRef","pageVersionId","contentPageId","title","pageText","publishedDate","sector","discipline","jobSector"]]
df["Digital"] = df.sector.apply(lambda x: "Digital" in x)
df["skill"] = df.title.apply(lambda x: getSkill(x))
df.to_excel("outputs/digital_jds.xlsx")
df.to_parquet("outputs/digital_jds.parquet.gzip",compression="gzip")
df

,jobRef,pageVersionId,contentPageId,title,pageText,publishedDate,sector,discipline,jobSector,Digital,skill
0,67558BR,5371358,80554,Principal Power Systems Studies Engineer,Principal Power Systems Studies Engineer About...,2023-11-06T00:00:00,[Energy],"[Electrical, Energy]",Energy,False,
1,67436BR,5371477,80484,Principal Mechanical Engineer,"About the roleOur Power, Process &amp; Nuclear...",2023-10-24T23:00:00,[Energy],[Energy],Energy,False,
2,65546BR,5371194,79294,Senior geospatial analyst,Senior geospatial analyst North America United...,2023-11-20T00:00:00,[Water and wastewater],"[GIS, Water]",Water,False,
3,66573BR,5371704,79819,Senior project information specialist,"Here at Mott MacDonald, we are experts in engi...",2023-09-11T23:00:00,"[Corporate services, Project management, Trans...","[Consultancy, Cost and quantity surveying, Pro...",Transport,False,
4,2255,5527660,83224,Senior Process Engineer,Senior Process Engineer Senior Process Enginee...,2024-05-13T15:12:09,[],"[Water, Civil]",Water,False,
...,...,...,...,...,...,...,...,...,...,...,...
1382,2639,5532267,83485,IT Specialist - Service Portfolio Management,IT Specialist - Service Portfolio Management I...,2024-05-29T08:12:06,[],[Information technology],Corporate services,False,
1383,67851BR,5371257,80744,Commercial Manager,"Commercial Manager Equality, diversity and inc...",2023-11-15T00:00:00,[Project management],"[Project controls, Project programme and comme...",Advisory,False,
1384,3298,5539165,83848,Active Travel Lead,Active Travel Lead Active Travel Lead Active T...,2024-06-28T11:39:25,[],[Planning and modelling],Transport,False,
1385,2103,5520297,83020,Senior Water/Wastewater Engineer,Senior Water/Wastewater Engineer Senior Water/...,2024-04-26T17:36:47,[],[Water],Water,False,
